#LOAD DATA WITH SPARK (25% of the directory 0)

In [1]:
sc.version

u'1.4.1'

In [2]:
data = sc.wholeTextFiles("test_bed_data")

In [3]:
from bs4 import BeautifulSoup
def parser(x):
    return BeautifulSoup(x, 'html.parser')

#Load Train CSV

In [30]:
train = gl.SFrame('train.csv')
t = list(train['file'])

PROGRESS: Finished parsing file /Users/ricardo/Documents/GuillermoGaps/spark-1.4.1/Kaggle_competition/train.csv
PROGRESS: Parsing completed. Parsed 100 lines in 0.107852 secs.
------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[str,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------
PROGRESS: Finished parsing file /Users/ricardo/Documents/GuillermoGaps/spark-1.4.1/Kaggle_competition/train.csv
PROGRESS: Parsing completed. Parsed 101107 lines in 0.072736 secs.


#Mapped the data using Beautifulsoup to get the text

In [4]:
mapped_data = data.map(lambda x: [x[0].split("/")[-1], parser(x[1]).get_text(), "train" if x[0].split("/")[-1] in t else "test"])

#Keep only data from the training set

In [5]:
mapped_train_data = mapped_data.filter( lambda x: x[2]=="train")

In [6]:
mapped_train_data.take(1)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 1 times, most recent failure: Lost task 0.0 in stage 0.0 (TID 0, localhost): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/ricardo/Documents/GuillermoGaps/spark-1.4.1/python/lib/pyspark.zip/pyspark/worker.py", line 111, in main
    process()
  File "/Users/ricardo/Documents/GuillermoGaps/spark-1.4.1/python/lib/pyspark.zip/pyspark/worker.py", line 106, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/Users/ricardo/Documents/GuillermoGaps/spark-1.4.1/python/lib/pyspark.zip/pyspark/serializers.py", line 263, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/Users/ricardo/Documents/GuillermoGaps/spark-1.4.1/python/pyspark/rdd.py", line 1273, in takeUpToNumLeft
    yield next(iterator)
  File "<ip

#Import the RDD to SFrame

In [2]:
import graphlab as gl

In [26]:
sf = gl.SFrame.from_rdd(mapped_train_data)

#ADD some features to the SFrame

In [66]:
sf['file'] = sf['X1'].apply(lambda x: x[0] )
sf['html'] = sf['X1'].apply(lambda x: x[1].strip().replace("  ", " " ))
sf['train'] = sf['X1'].apply(lambda x: x[2])

In [29]:
sf.head()

X1,files,html,train
"[1000188_raw_html.txt,\n\n\n\n\n The impact ...",1000188_raw_html.txt,The impact of shadow IT\n\n\n\n\n\n\n\n\n\n\n\ ...,train
"[1000662_raw_html.txt,\n\n\n\n\n\n ...",1000662_raw_html.txt,"(function() { var b=window,f=""chrome"",g=""tick"" ...",train
"[1000860_raw_html.txt,\n\n\n\n\n\n\n\n\n\n\n\n ...",1000860_raw_html.txt,J!NX : J!NX\n\n\n\n\n\n\n\n // ...,train
"[1001544_raw_html.txt,\n\n\n\n Dudes With ...",1001544_raw_html.txt,Dudes With Beards WithCats | Pictures of dudes ...,train
"[1001670_raw_html.txt,\n\n\n\n\n\n Thanks ...",1001670_raw_html.txt,Thanks for Voting - R1\n\n\n * {\n\tmargin: ...,train
"[1002222_raw_html.txt,\n\n\n\n snopes.com: ...",1002222_raw_html.txt,snopes.com: ProfessorIndrek Wichman E-mail\n ...,train
"[1002774_raw_html.txt,\n\n\n\n Mick Jagger ...",1002774_raw_html.txt,Mick Jagger sends upMonty Python before ...,train
"[100278_raw_html.txt,\n\n\n\n var ...",100278_raw_html.txt,varue_t0=window.ue_t0||+new ...,train
"[1002978_raw_html.txt,\n\n\n\n\n\n var ...",1002978_raw_html.txt,var _sf_startpt=(newDate()).getTime()\n ...,train
"[1003068_raw_html.txt,\n\n\n\n\n\n\n\n\n\n\n ...",1003068_raw_html.txt,How to Make Bacon InfusedVodka - Betty Crocker ...,train


#Adding a few more features

In [33]:
new_sf = sf.join(train,on=["file"], how="inner")

In [35]:
new_sf["length_html"] = new_sf['html'].apply(lambda x: len(x))

In [37]:
new_sf["bag_of_words"] = gl.text_analytics.count_words(new_sf['html'])

In [40]:
my_var = gl.text_analytics.tf_idf(gl.text_analytics.count_words(new_sf['html']))

In [42]:
new_sf["tfidf"] = my_var["docs"]

#Removing stop words for tfidf, bigrams and trigrams

In [44]:
stop_words = gl.text_analytics.stopwords()

In [45]:
def remove_sw(x):
    cleaned = []
    for element in x.split():
        if element.lower() not in stop_words:
            cleaned.append(element)
    
    return " ".join(cleaned)

new_sf['cleaned_html'] = new_sf['html'].apply(lambda x: remove_sw(x))
new_sf['trigrams'] = gl.text_analytics.count_ngrams(new_sf['cleaned_html'], 3, "word")
new_sf['bigrams'] = gl.text_analytics.count_ngrams(new_sf['cleaned_html'], 2, "word")

In [47]:
new_sf['stop_tfidf'] = new_sf['tfidf'].dict_trim_by_keys(stop_words, exclude=True)

#Counting Links, roughly

In [56]:
new_sf['https']= new_sf["html"].apply(lambda x: x.count("https"))

In [58]:
new_sf['http']= new_sf["html"].apply(lambda x: x.count("http:"))

#Counting "Sponsor content" string

In [60]:
new_sf['sponsor_content'] = new_sf["html"].apply(lambda x: x.lower().count("sponsor content"))

In [67]:
#Training logistic regression with 'stop_tfidf','bigrams', 'trigrams','length_html' as features

In [49]:
train_data, test_data = new_sf.random_split(0.8, seed=1234)
model_stop_tfidf = gl.logistic_classifier.create(train_data, target='sponsored', features = ['stop_tfidf','bigrams', 'trigrams','length_html'])
#predictions = model.predict(test_data)
results = model_stop_tfidf.evaluate(test_data)

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.

PROGRESS: Logistic regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 4285
PROGRESS: Number of classes           : 2
PROGRESS: Number of feature columns   : 4
PROGRESS: Number of unpacked features : 9367344
PROGRESS: Number of coefficients    : 9367345
PROGRESS: Starting L-BFGS
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+-----------+--------------+-------------------+---------------------+
PROGRESS: | Iteration | Passes   | Step size | Elapsed Time | Training-accuracy | Validation-accuracy |
PROGRESS: +-----------+----------+-----------+--------------+-------------------+---------------------+
PROGRESS: | 1         | 9        | 0.000006  | 26.483852    | 0.976429          | 0.905405            |

In [50]:
results

{'accuracy': 0.9051643192488263, 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      1       |        0        |   98  |
 |      1       |        1        |   22  |
 |      0       |        0        |  942  |
 |      0       |        1        |   3   |
 +--------------+-----------------+-------+
 [4 rows x 3 columns]}

#Adding new features  'https','http','sponsor_content'

In [64]:
train_data, test_data = new_sf.random_split(0.8, seed=1234)
model_stop_tfidf_2 = gl.logistic_classifier.create(train_data, target='sponsored', features = ['stop_tfidf','bigrams', 'trigrams','length_html','https','http','sponsor_content'])
#predictions = model.predict(test_data)
results_2 = model_stop_tfidf_2.evaluate(test_data)

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.

PROGRESS: Logistic regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 4272
PROGRESS: Number of classes           : 2
PROGRESS: Number of feature columns   : 7
PROGRESS: Number of unpacked features : 9389227
PROGRESS: Number of coefficients    : 9389228
PROGRESS: Starting L-BFGS
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+-----------+--------------+-------------------+---------------------+
PROGRESS: | Iteration | Passes   | Step size | Elapsed Time | Training-accuracy | Validation-accuracy |
PROGRESS: +-----------+----------+-----------+--------------+-------------------+---------------------+
PROGRESS: | 1         | 9        | 0.000006  | 19.568847    | 0.977060          | 0.927660            |

In [65]:
results_2

{'accuracy': 0.9042253521126761, 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      1       |        0        |   95  |
 |      1       |        1        |   25  |
 |      0       |        0        |  938  |
 |      0       |        1        |   7   |
 +--------------+-----------------+-------+
 [4 rows x 3 columns]}

#Checking for imbalanced data

In [54]:
model_stop_tfidf_class = gl.logistic_classifier.create(train_data, target='sponsored', features = ['stop_tfidf','bigrams', 'trigrams','length_html'], class_weights = 'auto')
results_class = model_stop_tfidf_class.evaluate(test_data)

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.

PROGRESS: Logistic regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 4269
PROGRESS: Number of classes           : 2
PROGRESS: Number of feature columns   : 4
PROGRESS: Number of unpacked features : 9459148
PROGRESS: Number of coefficients    : 9459149
PROGRESS: Starting L-BFGS
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+-----------+--------------+-------------------+---------------------+
PROGRESS: | Iteration | Passes   | Step size | Elapsed Time | Training-accuracy | Validation-accuracy |
PROGRESS: +-----------+----------+-----------+--------------+-------------------+---------------------+
PROGRESS: | 1         | 8        | 0.000017  | 19.466184    | 0.965566          | 0.663866            |

In [55]:
results_class

{'accuracy': 0.8704225352112676, 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      0       |        1        |   86  |
 |      0       |        0        |  859  |
 |      1       |        0        |   52  |
 |      1       |        1        |   68  |
 +--------------+-----------------+-------+
 [4 rows x 3 columns]}

#Trying Boosted Decision trees classifier

In [ ]:
model_trees = gl.boosted_trees_classifier.create(train_data, target='sponsored', features= ['stop_tfidf','bigrams', 'trigrams','length_html'])
#predicitons = model.classify(test_data)
results_trees = model_trees.evaluate(test_data)

In [53]:
results_trees

{'accuracy': 0.8985915492957747, 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      0       |        1        |   5   |
 |      0       |        0        |  940  |
 |      1       |        0        |  103  |
 |      1       |        1        |   17  |
 +--------------+-----------------+-------+
 [4 rows x 3 columns]}

#Another parser for later

In [27]:
from xml.etree.ElementTree import XMLParser

class MaxDepth:                     # The target object of the parser
     maxDepth = 0
     depth = 0
     def start(self, tag, attrib):   # Called for each opening tag.
         self.depth += 1
         if self.depth > self.maxDepth:
             self.maxDepth = self.depth
     def end(self, tag):             # Called for each closing tag.
         self.depth -= 1
     def data(self, data):
         pass            # We do not need to do anything with data.
     def close(self):    # Called when all data has been parsed.
         return self.maxDepth

def depth(exampleXml):
    target = MaxDepth()
    parser = XMLParser(target=target)
    parser.feed(exampleXml)
    return parser.close()